In [9]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Step 1: Define the problem (Noise Reduction)
np.random.seed(42)
original_image = np.zeros((50, 50))
noisy_image = original_image + np.random.normal(0, 0.3, (50, 50))  # Add Gaussian noise

# Step 2: Define parameters
iterations = 10
grid_size = noisy_image.shape
neighborhood = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]

# Step 3: Define function to update cell based on neighbors
def update_cell(i, j, grid):
    neighbors = []
    for dx, dy in neighborhood:
        ni, nj = i + dx, j + dy
        if 0 <= ni < grid_size[0] and 0 <= nj < grid_size[1]:
            neighbors.append(grid[ni, nj])
    return (grid[i, j] + np.mean(neighbors)) / 2

# Step 4: Parallel update function
def parallel_update(grid):
    new_grid = np.copy(grid)
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(grid_size[0]):
            for j in range(grid_size[1]):
                futures[executor.submit(update_cell, i, j, grid)] = (i, j)
        for future in futures:
            i, j = futures[future]
            new_grid[i, j] = future.result()
    return new_grid

# Step 5: Print header
print("--- Parallel Cellular Algorithm: Noise Reduction ---")
print(f"Initializing noisy image grid ({grid_size[0]}x{grid_size[1]})")
print(f"Iterations: {iterations}")
print("Neighborhood: 8 neighbors (Moore neighborhood)\n")

# Step 6: Iterative denoising process
current_grid = noisy_image
for t in range(1, iterations + 1):
    current_grid = parallel_update(current_grid)
    print(f"Processing iteration {t} ...")

# Step 7: Compute and print results
noise_before = np.mean(np.abs(noisy_image - original_image))
noise_after = np.mean(np.abs(current_grid - original_image))

print("\n--- Results ---")
print(f"Noise Level Before: {noise_before:.4f}")
print(f"Noise Level After : {noise_after:.4f}")

if noise_after < noise_before:
    print("Noise Reduction Achieved ✅")
else:
    print("No improvement detected ❌")

print("\nBest Solution: Smoothed (Denoised) Image Grid Found")
print(f"Convergence achieved after {iterations} iterations.")
print("---------------------------------------------------")


--- Parallel Cellular Algorithm: Noise Reduction ---
Initializing noisy image grid (50x50)
Iterations: 10
Neighborhood: 8 neighbors (Moore neighborhood)

Processing iteration 1 ...
Processing iteration 2 ...
Processing iteration 3 ...
Processing iteration 4 ...
Processing iteration 5 ...
Processing iteration 6 ...
Processing iteration 7 ...
Processing iteration 8 ...
Processing iteration 9 ...
Processing iteration 10 ...

--- Results ---
Noise Level Before: 0.2348
Noise Level After : 0.0346
Noise Reduction Achieved ✅

Best Solution: Smoothed (Denoised) Image Grid Found
Convergence achieved after 10 iterations.
---------------------------------------------------
